In [60]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn import svm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn import svm
import operator as op
import math
from hmmlearn import hmm
import pickle

In [61]:
modelscount = 50
nstates = 5

In [62]:
columnNames = ["DateTime",
"HipCenter X","HipCenter Y","HipCenter Z","HipCenter Depth",
"Spine X","Spine Y","Spine Z","Spine Depth",
"Head X","Head Y","Head Z","Head Depth",
"ShoulderCenter X","ShoulderCenter Y","ShoulderCenter Z","ShoulderCenter Depth",
"ShoulderLeft X","ShoulderLeft Y","ShoulderLeft Z","ShoulderLeft Depth",
"ElbowLeft X","ElbowLeft Y","ElbowLeft Z","ElbowLeft Depth",
"WristLeft X","WristLeft Y","WristLeft Z","WristLeft Depth",
"HandLeft X","HandLeft Y","HandLeft Z","HandLeft Depth",
"ShoulderRight X","ShoulderRight Y","ShoulderRight Z","ShoulderRight Depth",
"ElbowRight X","ElbowRight Y","ElbowRight Z","ElbowRight Depth",
"WristRight X","WristRight Y","WristRight Z","WristRight Depth",
"HandRight X","HandRight Y","HandRight Z","HandRight Depth",
"HipLeft X","HipLeft Y","HipLeft Z","HipLeft Depth",
"KneeLeft X","KneeLeft Y","KneeLeft Z","KneeLeft Depth",
"AnkleLeft X","AnkleLeft Y","AnkleLeft Z","AnkleLeft Depth",
"FootLeft X","FootLeft Y","FootLeft Z","FootLeft Depth",
"HipRight X","HipRight Y","HipRight Z","HipRight Depth",
"KneeRight X","KneeRight Y","KneeRight Z","KneeRight Depth",
"AnkleRight X","AnkleRight Y","AnkleRight Z","AnkleRight Depth",
"FootRight X","FootRight Y","FootRight Z","FootRight Depth"]

In [63]:
#Data Sets
classfication = ["not moving", "moving"]

fileWalkingSet = r"C:\Users\Orozc\OneDrive\My Documents\CPP\ME 6950\Test Set\Walking"
fileNotWalking = r"C:\Users\Orozc\OneDrive\My Documents\CPP\ME 6950\Test Set\Not Walking"

#data sets
##walking
walking1 = pd.read_csv(fileWalkingSet + "\\Waling 1.txt", names = columnNames)
walking2 = pd.read_csv(fileWalkingSet + "\\Waling 2.txt", names = columnNames)
walking3 = pd.read_csv(fileWalkingSet + "\\Waling 3.txt", names = columnNames)
walking4 = pd.read_csv(fileWalkingSet + "\\Waling 4.txt", names = columnNames)
walking5 = pd.read_csv(fileWalkingSet + "\\Waling 6.txt", names = columnNames)
walking6 = pd.read_csv(fileWalkingSet + "\\Waling7.txt", names = columnNames)
walking7 = pd.read_csv(fileWalkingSet + "\\Waling8.txt", names = columnNames)
walking8 = pd.read_csv(fileWalkingSet + "\\Waling9.txt", names = columnNames)
walking9 = pd.read_csv(fileWalkingSet + "\\Waling10.txt", names = columnNames)
walking10 = pd.read_csv(fileWalkingSet + "\\Waling11.txt", names = columnNames)


##notwalking
notwalking1 = pd.read_csv(fileNotWalking + "\\Standing1.txt", names = columnNames)
notwalking2 = pd.read_csv(fileNotWalking + "\\Standing2.txt", names = columnNames)
notwalking3 = pd.read_csv(fileNotWalking + "\\Standing3.txt", names = columnNames)
notwalking4 = pd.read_csv(fileNotWalking + "\\Standing4.txt", names = columnNames)
notwalking5 = pd.read_csv(fileNotWalking + "\\Standing5.txt", names = columnNames)
notwalking6 = pd.read_csv(fileNotWalking + "\\Standing6.txt", names = columnNames)
notwalking7 = pd.read_csv(fileNotWalking + "\\Standing7.txt", names = columnNames)


#dropping depth columns
##walking
walking1 = walking1.loc[:,~walking1.columns.str.endswith('Depth')]
walking2 = walking2.loc[:,~walking2.columns.str.endswith('Depth')]
walking3 = walking3.loc[:,~walking3.columns.str.endswith('Depth')]
walking4 = walking4.loc[:,~walking4.columns.str.endswith('Depth')]
walking5 = walking5.loc[:,~walking5.columns.str.endswith('Depth')]
walking6 = walking6.loc[:,~walking6.columns.str.endswith('Depth')]
walking7 = walking7.loc[:,~walking7.columns.str.endswith('Depth')]
walking8 = walking8.loc[:,~walking8.columns.str.endswith('Depth')]
walking9 = walking9.loc[:,~walking9.columns.str.endswith('Depth')]
walking10 = walking10.loc[:,~walking10.columns.str.endswith('Depth')]


##notwalking
notwalking1 = notwalking1.loc[:,~notwalking1.columns.str.endswith('Depth')]
notwalking2 = notwalking2.loc[:,~notwalking2.columns.str.endswith('Depth')]
notwalking3 = notwalking3.loc[:,~notwalking3.columns.str.endswith('Depth')]
notwalking4 = notwalking4.loc[:,~notwalking4.columns.str.endswith('Depth')]
notwalking5 = notwalking5.loc[:,~notwalking5.columns.str.endswith('Depth')]
notwalking6 = notwalking6.loc[:,~notwalking6.columns.str.endswith('Depth')]
notwalking7 = notwalking7.loc[:,~notwalking7.columns.str.endswith('Depth')]


#dropping Foot


#fixing the columnNames
for element in columnNames:
    if "Depth" in element:
        columnNames.remove(element)

#  Walking HMM model (walking)

In [74]:
#Creating the model
#data /Date Time was messing with results, drop rows with na values in any columns
x = [walking1.drop(columns=["DateTime"]).dropna(),walking2.drop(columns=["DateTime"]).dropna(),
     walking3.drop(columns=["DateTime"]).dropna(),
     walking4.drop(columns=["DateTime"]).dropna(), walking5.drop(columns=["DateTime"]).dropna(),
     walking6.drop(columns=["DateTime"]).dropna(),
    walking7.drop(columns=["DateTime"]).dropna()]

X_walking=[]
y=[]

number =[]
#break up the data and create an array of differences (get the average difference of 5 frames (5 rows))
for i in range(0,len(x)):
    a=0
    frames = 5 #getting the abs average of the frames
    #get the closest divisble number of a selected frame
    lengths = len(x[i]) - len(x[i])%frames#frames 
    for j in range(0,lengths-frames,frames): #change in pixel location
        a1 = abs(x[i].iloc[j]-x[i].iloc[j+1])
        a2 = abs(x[i].iloc[j+1]-x[i].iloc[j+2])
        a3 = abs(x[i].iloc[j+2]-x[i].iloc[j+3])
        a4 = abs(x[i].iloc[j+3]-x[i].iloc[j+4])
        a5 = abs(x[i].iloc[j+4]-x[i].iloc[j+5])
        #transpose the series, combine the series into one dataframe and grab the max values'
        a = (pd.concat([a1.to_frame().T,a2.to_frame().T,a3.to_frame().T,a4.to_frame().T,a5.to_frame().T])).max()
        a.reset_index(drop=True, inplace=True)
        X_walking.append(a) #max difference between five frames
        
        #number.append(i)
        number.append(len(a1))
        b = a
        a = 0
        if i<6:
           # y.append(classfication[1]) #1 - walking
            y.append([1])
        else:
           # y.append(classfication[0]) #2 - not walking
            y.append([0])
            
walkingModels =[]
walking_hidden_states = []
for i in range(0,modelscount):
    # Define the CategoricalHMM model
    walkingModel = hmm.GaussianHMM(n_components=nstates, n_iter=1).fit(X_walking)
    # Predict the hidden states
    walking_hidden_states =(walkingModel.predict(X_walking))
    walkingModels.append(walkingModel)
#prob = walkingModel.score(X_walking)
#prob_next_step = walkingModel.transmat_[hidden_states[99], :]
#prob_next_step

In [75]:
walkingModel.predict(X_walking)

array([3, 2, 0, 0, 0, 3, 1, 2, 0, 0, 2, 3, 3, 4, 0, 2, 2, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 2, 0, 0, 0,
       0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

# HMM (not walking)

In [65]:
#Creating the model
#data /Date Time was messing with results, drop rows with na values in any columns
x = [notwalking1.drop(columns=["DateTime"]).dropna(), notwalking2.drop(columns=["DateTime"]).dropna(), 
     notwalking3.drop(columns=["DateTime"]).dropna(),
     notwalking4.drop(columns=["DateTime"]).dropna(),notwalking5.drop(columns=["DateTime"]).dropna(),
     notwalking6.drop(columns=["DateTime"]).dropna(),notwalking7.drop(columns=["DateTime"]).dropna()]

X_notwalking=[]
y=[]

number =[]
#break up the data and create an array of differences (get the average difference of 5 frames (5 rows))
for i in range(0,len(x)):
    a=0
    frames = 5 #getting the abs average of the frames
    #get the closest divisble number of a selected frame
    lengths = len(x[i]) - len(x[i])%frames#frames 
    for j in range(0,lengths-frames,frames): #change in pixel location
        a1 = abs(x[i].iloc[j]-x[i].iloc[j+1])
        a2 = abs(x[i].iloc[j+1]-x[i].iloc[j+2])
        a3 = abs(x[i].iloc[j+2]-x[i].iloc[j+3])
        a4 = abs(x[i].iloc[j+3]-x[i].iloc[j+4])
        a5 = abs(x[i].iloc[j+4]-x[i].iloc[j+5])
        #transpose the series, combine the series into one dataframe and grab the max values'
        a = (pd.concat([a1.to_frame().T,a2.to_frame().T,a3.to_frame().T,a4.to_frame().T,a5.to_frame().T])).max() 
        a.reset_index(drop=True, inplace=True)
        X_notwalking.append(a) #max difference between five frames

        if i<6:
           # y.append(classfication[1]) #1 - walking
            y.append([1])
        else:
           # y.append(classfication[0]) #2 - not walking
            y.append([0])
            
notwalkingModels = []
notwalking_hidden_states = []
for i in range(0,modelscount):
    # Define the CategoricalHMM model
    notwalkingModel = hmm.GaussianHMM(n_components=nstates, n_iter=1).fit(X_notwalking)
    # Predict the hidden states
    notwalking_hidden_states=notwalkingModel.predict(X_notwalking)
    notwalkingModels.append(notwalkingModel)


# Open Trial 

In [66]:
#Test with other data sets

columnNames = ["DateTime",
"HipCenter X","HipCenter Y","HipCenter Z","HipCenter Depth",
"Spine X","Spine Y","Spine Z","Spine Depth",
"Head X","Head Y","Head Z","Head Depth",
"ShoulderCenter X","ShoulderCenter Y","ShoulderCenter Z","ShoulderCenter Depth",
"ShoulderLeft X","ShoulderLeft Y","ShoulderLeft Z","ShoulderLeft Depth",
"ElbowLeft X","ElbowLeft Y","ElbowLeft Z","ElbowLeft Depth",
"WristLeft X","WristLeft Y","WristLeft Z","WristLeft Depth",
"HandLeft X","HandLeft Y","HandLeft Z","HandLeft Depth",
"ShoulderRight X","ShoulderRight Y","ShoulderRight Z","ShoulderRight Depth",
"ElbowRight X","ElbowRight Y","ElbowRight Z","ElbowRight Depth",
"WristRight X","WristRight Y","WristRight Z","WristRight Depth",
"HandRight X","HandRight Y","HandRight Z","HandRight Depth",
"HipLeft X","HipLeft Y","HipLeft Z","HipLeft Depth",
"KneeLeft X","KneeLeft Y","KneeLeft Z","KneeLeft Depth",
"AnkleLeft X","AnkleLeft Y","AnkleLeft Z","AnkleLeft Depth",
"FootLeft X","FootLeft Y","FootLeft Z","FootLeft Depth",
"HipRight X","HipRight Y","HipRight Z","HipRight Depth",
"KneeRight X","KneeRight Y","KneeRight Z","KneeRight Depth",
"AnkleRight X","AnkleRight Y","AnkleRight Z","AnkleRight Depth",
"FootRight X","FootRight Y","FootRight Z","FootRight Depth"]

## Not blind
PredictionFileLocation = r"C:\Users\Orozc\OneDrive\My Documents\CPP\ME 6950\SVM Models\prediction\not blind"
#Data Sets
test1 = pd.read_csv(PredictionFileLocation + "\\notwalking_Pred1.txt", names = columnNames)
test2 = pd.read_csv(PredictionFileLocation + "\\notwalking_Pred2.txt", names = columnNames)
test3 = pd.read_csv(PredictionFileLocation + "\\notwalking_Pred3.txt", names = columnNames)
test4 = pd.read_csv(PredictionFileLocation + "\\notwalking_Pred4.txt", names = columnNames)
test5 = pd.read_csv(PredictionFileLocation + "\\notwalking_Pred5.txt", names = columnNames)

test6 = pd.read_csv(PredictionFileLocation + "\\walking_Pred1.txt", names = columnNames)
test7 = pd.read_csv(PredictionFileLocation + "\\walking_Pred2.txt", names = columnNames)
test8 = pd.read_csv(PredictionFileLocation + "\\walking_Pred3.txt", names = columnNames)
test9 = pd.read_csv(PredictionFileLocation + "\\walking_Pred4.txt", names = columnNames)
test10 = pd.read_csv(PredictionFileLocation + "\\walking_Pred5.txt", names = columnNames)


#dropping depth columns
##not walking
test1 = test1.loc[:,~test1.columns.str.endswith('Depth')]
test2 = test2.loc[:,~test2.columns.str.endswith('Depth')]
test3 = test3.loc[:,~test3.columns.str.endswith('Depth')]
test4 = test4.loc[:,~test4.columns.str.endswith('Depth')]
test5 = test5.loc[:,~test5.columns.str.endswith('Depth')]

##walking
test6 = test6.loc[:,~test6.columns.str.endswith('Depth')]
test7 = test7.loc[:,~test7.columns.str.endswith('Depth')]
test8 = test8.loc[:,~test8.columns.str.endswith('Depth')]
test9 = test9.loc[:,~test9.columns.str.endswith('Depth')]
test10 = test10.loc[:,~test10.columns.str.endswith('Depth')]



#dropping depth columns
##walking
test1 = test1.loc[:,~test1.columns.str.endswith('Depth')]
test2 = test2.loc[:,~test2.columns.str.endswith('Depth')]
test3 = test3.loc[:,~test3.columns.str.endswith('Depth')]
test4 = test4.loc[:,~test4.columns.str.endswith('Depth')]
test5 = test5.loc[:,~test5.columns.str.endswith('Depth')]
test6 = test6.loc[:,~test6.columns.str.endswith('Depth')]
test7 = test7.loc[:,~test7.columns.str.endswith('Depth')]
test8 = test8.loc[:,~test8.columns.str.endswith('Depth')]
test9 = test9.loc[:,~test9.columns.str.endswith('Depth')]
test10 = test10.loc[:,~test10.columns.str.endswith('Depth')]

#fixing the columnNames
for element in columnNames:
    if "Depth" in element:
        columnNames.remove(element)
#

In [67]:
#Feed the data
x_pred = [test1.drop(columns=["DateTime"]).dropna(),test2.drop(columns=["DateTime"]).dropna(),
          test3.drop(columns=["DateTime"]).dropna(),
          test4.drop(columns=["DateTime"]).dropna(),test5.drop(columns=["DateTime"]).dropna(),
          test6.drop(columns=["DateTime"]).dropna(),test7.drop(columns=["DateTime"]).dropna(),
          test8.drop(columns=["DateTime"]).dropna(),test9.drop(columns=["DateTime"]).dropna(),
          test10.drop(columns=["DateTime"]).dropna()]
X_pred = []
y_pred = []
for i in range(0,len(x_pred)):
    a=0
    frames = 5 #getting the abs average of the frames
    #get the closest divisble number of a selected frame
    lengths = len(x_pred[i]) - len(x_pred[i])%frames#frames 
    for j in range(0,lengths-frames,frames): #change in pixel location
        a=0

        a1 = abs(x_pred[i].iloc[j]-x_pred[i].iloc[j+1])
        a2 = abs(x_pred[i].iloc[j+1]-x_pred[i].iloc[j+2])
        a3 = abs(x_pred[i].iloc[j+2]-x_pred[i].iloc[j+3])
        a4 = abs(x_pred[i].iloc[j+3]-x_pred[i].iloc[j+4])
        a5 = abs(x_pred[i].iloc[j+4]-x_pred[i].iloc[j+5])
        a = (pd.concat([a1.to_frame().T,a2.to_frame().T,a3.to_frame().T,a4.to_frame().T,a5.to_frame().T])).max()
        
        X_pred.append(a) #average difference of the five frames
       
        if i>4:
            y_pred.append(classfication[1]) #1 - walking
        else:
            y_pred.append(classfication[0]) #2 - not walkin
            
# Predict the hidden states
#notwalking_hidden_states = notwalkingModel.predict(X_pred)
#notwalking_prob = notwalkingModel.score(X_pred)

# Predict the hidden states
#walking_hidden_states = walkingModel.predict(X_pred)
#walking_prob = walkingModel.score(X_pred)



notwalking_prob_next_step_list = []
walking_prob_next_step_list = []
for j in range(0, modelscount):
    #print(i)
    walking_prob_next_step = walkingModels[j].transmat_[walkingModels[j].predict(X_pred), walkingModels[j].predict(X_pred)]
    notwalking_prob_next_step = notwalkingModels[j].transmat_[notwalkingModels[j].predict(X_pred), notwalkingModels[j].predict(X_pred)]
    walking_prob_next_step_list.append(walking_prob_next_step)
    notwalking_prob_next_step_list.append(notwalking_prob_next_step)
#print(max(walkingprob_list),max(notwalkingprob_list))


HMM_predictions = []
walkingprob_list = []
notwalkingprob_list = []
walking_step_probablity = []
notwalking_step_probablity = []    
for i in range(0, len(X_pred)):
    a=[]
    b=[]
    for j in range(0, modelscount):
        a.append(notwalking_prob_next_step_list[j][i])
        b.append(walking_prob_next_step_list[j][i])
    notwalking_step_probablity.append(max(a))
    walking_step_probablity.append(max(b))
    notwalkingprob=(max(a))
    walkingprob=(max(b))
    #print(notwalkingprob,walkingprob)
    if walkingprob > notwalkingprob:
        HMM_predictions.append(classfication[1])#1 - walking
    else:
        HMM_predictions.append(classfication[0])#0 - ot walking


In [68]:
# compare HMM predictions vs actual labels
correct_walking = 0 #correct 
correct_notwalking = 0 #correct
walking_num = 0 #track of overall count
notwalking_num = 0  #tack overall count
predicted_walking = 0
predicted_notwalking=0
for i in range (0, len(HMM_predictions)):
    if y_pred[i] == "not moving":
        notwalking_num +=1
        
        if y_pred[i] == HMM_predictions[i]:
            predicted_notwalking +=1
            correct_notwalking+=1
        else:
            predicted_walking+=1
    else:
        walking_num +=1
        if y_pred[i] == HMM_predictions[i]:
            correct_walking+=1
            predicted_walking+=1
        else:
            predicted_notwalking+=1
    
print("Walking Count", walking_num)       
print("Not Walking Count", notwalking_num) 

print("HMM Walking Count (Predict)", predicted_walking)       
print("HMM Not Walking Count (Predict)", predicted_notwalking) 

print("HMM Walking Count (Correct)", correct_walking)       
print("HMM Not Walking Count (Correct)", correct_notwalking) 
print("Accuraccy: ", ((correct_walking+correct_notwalking)/(walking_num+notwalking_num))*100)

Walking Count 367
Not Walking Count 910
HMM Walking Count (Predict) 931
HMM Not Walking Count (Predict) 346
HMM Walking Count (Correct) 359
HMM Not Walking Count (Correct) 338
Accuraccy:  54.581049334377454


# Notes

# import numpy as np
from hmmlearn import hmm

# Define the observations
observations = np.array([[1, 0, 1, 0, 1], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]).reshape(-1, 1)

# Define the lengths of the sequences
lengths = [5, 5, 5]

# Define the CategoricalHMM model
model = hmm.CategoricalHMM(n_components=2)

# Fit the model to the observations
model = model.fit(observations, lengths)

# Predict the hidden states
hidden_states = model.predict(observations)

# Get the probability of the observations
prob = model.score(observations, lengths)

# Print the hidden states and the probability
print("Hidden States: ", hidden_states)
print("Probability of Observations: ", prob)

#  Saving models

In [69]:
#with open("walkingModel.pkl", "wb") as file: pickle.dump(walkingModels, file)
#with open("notwalkingModel.pkl", "wb") as file: pickle.dump(notwalkingModels, file)


#  Loading Models

In [70]:
with open("walkingModel.pkl", "rb") as file: walkingmodels = pickle.load(file)
with open("notwalkingModel.pkl", "rb") as file: notwalkingmodels =pickle.load(file)

In [71]:
"""
classfication = ["not moving", "moving"]

fileWalkingSet = r"C:\Users\Orozc\OneDrive\My Documents\CPP\ME 6950\Test Set\Walking"
fileNotWalking = r"C:\Users\Orozc\OneDrive\My Documents\CPP\ME 6950\Test Set\Not Walking"

#data sets
##walking
walking1 = pd.read_csv(fileWalkingSet + "\\Waling 1.txt", names = columnNames)
walking2 = pd.read_csv(fileWalkingSet + "\\Waling 2.txt", names = columnNames)
walking3 = pd.read_csv(fileWalkingSet + "\\Waling 3.txt", names = columnNames)
walking4 = pd.read_csv(fileWalkingSet + "\\Waling 4.txt", names = columnNames)
walking5 = pd.read_csv(fileWalkingSet + "\\Waling 6.txt", names = columnNames)
walking6 = pd.read_csv(fileWalkingSet + "\\Waling7.txt", names = columnNames)
walking7 = pd.read_csv(fileWalkingSet + "\\Waling8.txt", names = columnNames)
walking8 = pd.read_csv(fileWalkingSet + "\\Waling9.txt", names = columnNames)
walking9 = pd.read_csv(fileWalkingSet + "\\Waling10.txt", names = columnNames)
walking10 = pd.read_csv(fileWalkingSet + "\\Waling11.txt", names = columnNames)


##notwalking
notwalking1 = pd.read_csv(fileNotWalking + "\\Standing1.txt", names = columnNames)
notwalking2 = pd.read_csv(fileNotWalking + "\\Standing2.txt", names = columnNames)
notwalking3 = pd.read_csv(fileNotWalking + "\\Standing3.txt", names = columnNames)
notwalking4 = pd.read_csv(fileNotWalking + "\\Standing4.txt", names = columnNames)
notwalking5 = pd.read_csv(fileNotWalking + "\\Standing5.txt", names = columnNames)
notwalking6 = pd.read_csv(fileNotWalking + "\\Standing6.txt", names = columnNames)
notwalking7 = pd.read_csv(fileNotWalking + "\\Standing7.txt", names = columnNames)


#dropping depth columns
##walking
walking1 = walking1.loc[:,~walking1.columns.str.endswith('Depth')]
walking2 = walking2.loc[:,~walking2.columns.str.endswith('Depth')]
walking3 = walking3.loc[:,~walking3.columns.str.endswith('Depth')]
walking4 = walking4.loc[:,~walking4.columns.str.endswith('Depth')]
walking5 = walking5.loc[:,~walking5.columns.str.endswith('Depth')]
walking6 = walking6.loc[:,~walking6.columns.str.endswith('Depth')]
walking7 = walking7.loc[:,~walking7.columns.str.endswith('Depth')]
walking8 = walking8.loc[:,~walking8.columns.str.endswith('Depth')]
walking9 = walking9.loc[:,~walking9.columns.str.endswith('Depth')]
walking10 = walking10.loc[:,~walking10.columns.str.endswith('Depth')]


##notwalking
notwalking1 = notwalking1.loc[:,~notwalking1.columns.str.endswith('Depth')]
notwalking2 = notwalking2.loc[:,~notwalking2.columns.str.endswith('Depth')]
notwalking3 = notwalking3.loc[:,~notwalking3.columns.str.endswith('Depth')]
notwalking4 = notwalking4.loc[:,~notwalking4.columns.str.endswith('Depth')]
notwalking5 = notwalking5.loc[:,~notwalking5.columns.str.endswith('Depth')]
notwalking6 = notwalking6.loc[:,~notwalking6.columns.str.endswith('Depth')]
notwalking7 = notwalking7.loc[:,~notwalking7.columns.str.endswith('Depth')]


#dropping Foot


#fixing the columnNames
for element in columnNames:
    if "Depth" in element:
        columnNames.remove(element)
#Feed the data
x_pred = [test1.drop(columns=["DateTime"]).dropna(),test2.drop(columns=["DateTime"]).dropna(),
          test3.drop(columns=["DateTime"]).dropna(),
          test4.drop(columns=["DateTime"]).dropna(),test5.drop(columns=["DateTime"]).dropna(),
          test6.drop(columns=["DateTime"]).dropna(),test7.drop(columns=["DateTime"]).dropna(),
          test8.drop(columns=["DateTime"]).dropna(),test9.drop(columns=["DateTime"]).dropna(),
          test10.drop(columns=["DateTime"]).dropna()]
X_pred = []
y_pred = []
for i in range(0,len(x_pred)):
    a=0
    frames = 5 #getting the abs average of the frames
    #get the closest divisble number of a selected frame
    lengths = len(x_pred[i]) - len(x_pred[i])%frames#frames 
    for j in range(0,lengths-frames,frames): #change in pixel location
        a=0

        a1 = abs(x_pred[i].iloc[j]-x_pred[i].iloc[j+1])
        a2 = abs(x_pred[i].iloc[j+1]-x_pred[i].iloc[j+2])
        a3 = abs(x_pred[i].iloc[j+2]-x_pred[i].iloc[j+3])
        a4 = abs(x_pred[i].iloc[j+3]-x_pred[i].iloc[j+4])
        a5 = abs(x_pred[i].iloc[j+4]-x_pred[i].iloc[j+5])
        a = (pd.concat([a1.to_frame().T,a2.to_frame().T,a3.to_frame().T,a4.to_frame().T,a5.to_frame().T])).max()
        
        X_pred.append(a) #average difference of the five frames
       
        if i>4:
            y_pred.append(classfication[1]) #1 - walking
        else:
            y_pred.append(classfication[0]) #2 - not walkin
            
# Predict the hidden states
#notwalking_hidden_states = notwalkingModel.predict(X_pred)
#notwalking_prob = notwalkingModel.score(X_pred)

# Predict the hidden states
#walking_hidden_states = walkingModel.predict(X_pred)
#walking_prob = walkingModel.score(X_pred)



notwalking_prob_next_step_list = []
walking_prob_next_step_list = []
for j in range(0, modelscount):
    #print(i)
    walking_prob_next_step = walkingmodels[j].transmat_[walkingmodels[j].predict(X_pred), walkingmodels[j].predict(X_pred)]
    notwalking_prob_next_step = notwalkingmodels[j].transmat_[notwalkingmodels[j].predict(X_pred), notwalkingmodels[j].predict(X_pred)]
    walking_prob_next_step_list.append(walking_prob_next_step)
    notwalking_prob_next_step_list.append(notwalking_prob_next_step)
#print(max(walkingprob_list),max(notwalkingprob_list))


HMM_predictions = []
walkingprob_list = []
notwalkingprob_list = []
walking_step_probablity = []
notwalking_step_probablity = []    
for i in range(0, len(X_pred)):
    a=[]
    b=[]
    for j in range(0, modelscount):
        a.append(notwalking_prob_next_step_list[j][i])
        b.append(walking_prob_next_step_list[j][i])
    notwalking_step_probablity.append(max(a))
    walking_step_probablity.append(max(b))
    notwalkingprob=(max(a))
    walkingprob=(max(b))
    #print(notwalkingprob,walkingprob)
    if walkingprob > notwalkingprob:
        HMM_predictions.append(classfication[1])#1 - walking
    else:
        HMM_predictions.append(classfication[0])#0 - ot walking
        
        
# compare HMM predictions vs actual labels
correct_walking = 0 #correct 
correct_notwalking = 0 #correct
walking_num = 0 #track of overall count
notwalking_num = 0  #tack overall count
predicted_walking = 0
predicted_notwalking=0

for i in range (0, len(HMM_predictions)):
    if y_pred[i] == "not moving":
        notwalking_num +=1
        
        if y_pred[i] == HMM_predictions[i]:
            predicted_notwalking +=1
            correct_notwalking+=1
        else:
            predicted_walking+=1
    else:
        walking_num +=1
        if y_pred[i] == HMM_predictions[i]:
            correct_walking+=1
            predicted_walking+=1
        else:
            predicted_notwalking+=1
    
print("Walking Count", walking_num)       
print("Not Walking Count", notwalking_num) 

print("HMM Walking Count (Predict)", predicted_walking)       
print("HMM Not Walking Count (Predict)", predicted_notwalking) 

print("HMM Walking Count (Correct)", correct_walking)       
print("HMM Not Walking Count (Correct)", correct_notwalking) 
print("Accuraccy: ", ((correct_walking+correct_notwalking)/(walking_num+notwalking_num))*100)
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 64-65: truncated \UXXXXXXXX escape (4218438092.py, line 170)